In [1]:
import numpy  as np
import pandas as pd

import itertools
import random
import math  
import copy

import pandas_profiling

# Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
%matplotlib inline  
import seaborn as sns
sns.set(style="white")

# scikit-learn
from sklearn.dummy           import DummyClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier
from sklearn.ensemble        import GradientBoostingClassifier

# Supporting functions from scikit-learn
from sklearn.metrics         import confusion_matrix
from sklearn.metrics         import roc_curve
from sklearn.metrics         import roc_auc_score
from sklearn.pipeline        import Pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree            import export_graphviz
from sklearn.decomposition   import PCA

# for text processing
import spacy
import re
import nltk
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Set a seed for replication
SEED = 1  # Use this anywhere a stochastic function allows you to set a seed

In [2]:
df_learn=pd.read_excel("IPO_data_to_learn.xlsx")
df_predict=pd.read_excel("IPO_data_to_predict.xlsx")

# Quick look at the data

In [3]:
df_learn.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3330 entries, 0 to 3329
Data columns (total 47 columns):
Unnamed: 0            3330 non-null int64
closeDay1             3215 non-null float64
offerPrice            3330 non-null float64
rf                    3057 non-null object
egc                   3330 non-null bool
html                  3330 non-null bool
patRatio              2055 non-null float64
city                  3329 non-null object
issuer                3330 non-null object
highTech              3330 non-null bool
age                   3148 non-null float64
exchange              3330 non-null object
year                  3330 non-null int64
industryFF5           3330 non-null object
industryFF12          3330 non-null object
industryFF48          3330 non-null object
nUnderwriters         3330 non-null int64
sharesOfferedPerc     3068 non-null float64
totalProceeds         3330 non-null int64
manager               3330 non-null object
investmentReceived    1830 non-null fl

**Comments**: We observe that there 47 columns out of which 39 are numeric/bool and 8 are text type. We also notice that there are some missing data in many fields. 

Now we look at the detailed profile of the dataFrame using pandas profiling. (The report is provided in HTML along with the submission. Observations related to the report are given below).

In [4]:
#file.profile_report()
profile = pandas_profiling.ProfileReport(df_learn)
profile.to_file(outputfile='learn_data_profile_report.html')

**Observations from the profile report**

* Presence of missing values (Either drop or process the missig fileds)
* High correlation among 5 fields (May be ignored or Use PCA to reduce the dimentionality)
* Different scales, ranging from 0 to 1e9. (Need to normalize the data)
* Missing outcome: offerPrice(3.5%) and closeDay1(3.5%). Since there is no outcome, it may not be useful to use this data, may be dropped.
* Only 22% of the companies are marked emerging growth companies. (Possibility of bias?)
* Most of the companies are listed in NASDAQ(2368), followed by NYSE(895)
* Data is present from 1996 to 2018 (More data in the late 90s, but data is well spread across years)
* Five fields that are skewed (totalProceeds, InvestmentReceived, commonEquity1, totalRevenue, nPatents)
* 19 fileds out of 47 have missing entries. Highest missing entries in investmentReceived(45%) followed by nExecutives, priorFinancing, nVCs, patRatio, managementFee(32.9%) in descending order

In [5]:
# Profile report for the prediction data
profile = pandas_profiling.ProfileReport(df_predict)
profile.to_file(outputfile='predict_data_profile_report.html')

In [6]:
df_learn['exchange'].value_counts()

NASDQ    2368
NYSE      895
AMEX       67
Name: exchange, dtype: int64

In [7]:
df_learn['industryFF5'].value_counts()

Business Equipment, Telephone and Television Transmission                                         1122
Other                                                                                              898
Healthcare, Medical Equipment, and Drugs                                                           621
Consumer Durables, NonDurables, Wholesale, Retail, and Some Services (Laundries, Repair Shops)     393
Manufacturing, Energy, and Utilities                                                               296
Name: industryFF5, dtype: int64

In [8]:
df_learn['industryFF12'].value_counts().head()

Business Equipment -- Computers, Software, and Electronic Equipment    945
Healthcare, Medical Equipment, and Drugs                               621
Finance                                                                483
Other                                                                  462
Wholesale, Retail, and Some Services (Laundries, Repair Shops)         271
Name: industryFF12, dtype: int64

In [9]:
df_learn['industryFF48'].value_counts().head()

Business Services          845
Pharmaceutical Products    409
Trading                    221
Electronic Equipment       190
Retail                     158
Name: industryFF48, dtype: int64

**Comments**: Will probably need to only keep 1 of these, maybe FF12?


In [32]:
df_learn = df_learn.rename(columns={"commonEquity":"commonEquity1","commonEquity.1":"commonEquity2" })

**Comments**: Age is the age of the firm. Should replace mising values by the mean? Are the zeros indicating new firms?

In [33]:
print(df_learn['age'].mean())
print(df_learn['age'].mode())

16.184561626429478
0    3.0
dtype: float64


In [8]:
len([w for w in file['age'] if w>100])

53

**Comments**: Is it possible to have so much old companies?

In [10]:
print(file['blueSky'].median(), file['blueSky'].mean())

10000.0 12916.115884115885


**Comments**: Should do something with *blue sky* missing value: need to decide if really useful and, if it is, how to fill it. mean, median or mean between 1sr and 3rd quarter

# Text pre-process on Risk Factor

In [34]:
# Get English Stopwords from NLTK & Extend the stop word list
SW = stopwords.words('english')
SW.append('risk')
SW.append('factors')


In [35]:
#looks how many percent of risk facor are NaN values
df_learn['rf'].isna().sum()/len(df_learn['rf'])

0.08198198198198198

In [36]:
#file them with a blank
df_learn['rf'] = df_learn['rf'].fillna('')

In [37]:
# Define a custom function to clean punctuation from  given text
def clean_punctuation(txt):
    return str(txt).replace(",", " ").replace(".", " ").replace(":", " ").replace(";", " ").replace("?", " ").replace("!", " ").replace("\\", " ")


In [38]:
# Apply the function to clean the rf column
df_learn['rf'] = df_learn['rf'].apply(clean_punctuation)    
df_learn['rf'].head(1)

0    RISK FACTORS You should carefully consider the...
Name: rf, dtype: object

In [40]:
# Define a custom function to clean some given text
def clean_re(txt):
    txt = re.sub("[+]", " ", txt)
    txt = re.sub("[&]", " ", txt)
    txt = re.sub("\s \s", " ", txt)
    txt = re.sub("\s \s \s", " ", txt)
    return txt

In [41]:
# Apply clean_re() to all features
df_learn['rf'] = df_learn['rf'].apply(clean_re)    
df_learn['rf'].head(1)

0    RISK FACTORS You should carefully consider the...
Name: rf, dtype: object

In [42]:
# Define custom function to remove stopwords
def clean_sw(txt):
    li = list(txt.lower().split(" "))
    txt = [word for word in li if word not in SW]
    return ' '.join(txt)

In [43]:
# Apply function to remove stopwords
df_learn['rf'] = df_learn['rf'].apply(clean_sw)    
df_learn['rf'].head(1)

0    carefully consider following information conta...
Name: rf, dtype: object

In [45]:
# Define custom function to wrap simple_preprocess() from gensim
def sp(txt):
    return simple_preprocess(txt)

In [46]:
# Apply simple_preprocess() to all features
df_learn['rf'] = df_learn['rf'].apply(sp)    
df_learn['rf'].head(1)

0    [carefully, consider, following, information, ...
Name: rf, dtype: object

In [47]:
#nltk.download('wordnet') #downloading wordnet to use lemmatizer
lmtz=nltk.stem.WordNetLemmatizer()
# Write a lemmatization function based on nltk.stem.WordNetLemmatizer()
def lemmatiz(txt):
    return [lmtz.lemmatize(mot) for mot in txt]

In [48]:
# Apply lemmatize_text() to all features  
df_learn['rf'] = df_learn['rf'].apply(lemmatiz)    
df_learn['rf'].head(1)

0    [carefully, consider, following, information, ...
Name: rf, dtype: object

# Preprocess of data


In [5]:
data = copy.copy(df_learn)

In [6]:
data['age'].fillna(data['age'].median(), inplace=True)

In [7]:
data['blueSky'].fillna(data['blueSky'].median(), inplace=True)

We decide to drop the data corresponding to missing or negative values of 'closeDay1' as this feature is very important and so replacing missing values may lead to huge misinterpretation. Moreover it corresponds to only 3.5% of the data


In [8]:
data.dropna(subset=["closeDay1"], inplace=True)

In [9]:
data.shape

(3215, 47)

#data.profile_report()

Referring to the profile report generated, attached as HTML along with submission.

We can see that some features are highly correlated, so we will use PCA to remove the redundant features.